# Biweekly Report 3 
## Understanding and Implementing the BLEU Score Metric

### Logan Barnhart

In this report we'll be building some intuition around the BLEU score and implementing it.

The BLEU score is an incredibly important metric in the field of machine translation because it served as a baseline sense of accuracy across any given model which would prove valuable when seeing if newer models indeed improved upon older ones. 

The original paper can be found [here](https://aclanthology.org/P02-1040.pdf)

### Why is BLEU needed over a simple accuracy metric?

The primary issue researchers faced was that there are often times many correct translations of any given sentence, so a new metric was needed to adequately handle all the different cases. 

Additionally, all of the metrics researchers were working with until BLEU was implemented were incredibly expensive and slowed any progress to a hault. 

## The big idea behind BLEU

Typically, the many different but ___on average___ correct translations will share a common vocabulary and some similarity in ordering. The idea is to break the machine translation and reference translations into several sets of n-grams and calculate a weighted average of accuracy across the sets. 

It turns out that BLEU is not perfect, and may make non-intuitive scores on individual  examples, but on average it makes intuitive scores that would match with a human scorer. 


# Formula - Part 1

The researchers use a modified precision which uses the following formula:

The score when considering n-gram words in a single sentence, C is: 

#$p_n = \frac{\sum_{\text{n-gram} \in C} \textrm{Count}_{clip}(\text{n-gram})}{\sum_{\text{n-gram} \in C} \textrm{Count}(\text{n-gram})}$

where 
* $\textrm{Count}(\text{n-gram})$ is the number of times n-gram occurs in $C$
* $\textrm{Count}_{clip} = \min \left ( \textrm{Count}(\text{n-gram}), \textrm{Count}_{ref}(\text{n-gram}) \right)$ 

and 

* $\textrm{Count}_{ref}(\text{n-gram})$ is the largest number of times n-gram occurs in any of the reference sentences. 

## Example

Let's say we are given a sentence in Chinese that translates to some variation of "The cat is on the mat."

Machine translation: 

* The cat the cat the cat.

Reference translations:

* The cat is on the mat.
* There is a cat on the mat.

If we're considering unigrams, then 

$p_1 = \frac{2 + 1}{3 + 3} = \frac{1}{2}$

since 'the' occurs 2 times in the first reference, 'cat' occurs 1 time in either of the reference sentences, and 'the' and 'cat' both occur 3 times in the translation.

Considering bigrams, then 

$p_2 = \frac{1}{3 + 2} = \frac{1}{5}$ since 'the cat' only occurs 1 time in the first reference with 'cat the' not occuring in either reference, while 'the cat' occurs 3 times in the translation, and 'cat the' occurs 2 times in the translation.



# Formula - Part 2

The above method will work on individual sentences, but generally machines will be translating numerous sentences or entire documents. To resolve this the researchers expand the formula by letting 

#$p_n = \frac{\sum_{C \in \{sentences\}} \sum_{\text{n-gram} \in C} \textrm{Count}_{clip}(\text{n-gram})}{ \sum_{C \in \{ sentences \}} \sum_{\text{n-gram} \in C} \textrm{Count}(\text{n-gram})}$

Where $C$ is a sentence in the set of machine translation sentences. Loosely speaking, this is just a way to average the accuracies of each sentence.





# Formula - Part 3
The researchers wanted a single formula to consider the accuracy across different length n-grams, but while exploring numerically they found a few tendencies of the current scoring system:
* shorter sentences tended to score higher even when translations were inaccurate
* The score, $p_n$, decayed (roughly) exponentially as n increased

To address both issues, they introduced a 'brevity penalty' (BP) and would do a weighted sum of the logs of the scores across several n-grams, because of this their final formula is:

# $\textrm{BLEU} = \textrm{BP} \cdot \exp \left( \sum_{n = 1}^N w_n \log(p_n) \right)$

To calculate BP, we let 

* $c$ be the length of the translation 

and 

* $r$ be the 'effective reference length' which is calculated by summing the lengths of the reference sentences that are closest in length to the machine translation they would correlate to 

(i.e. if our machine-translated document was "Hello. Goodbye." and our set of reference translations for the first and second sentences respectively was {"Hi.", "Hey There."} and {"See you.", "Until next time."} then $c = 2$ and $r = 1 + 2$)

Then

#$BP = 
  \begin{cases}
    1 & \text{if } c > r\\
    \exp(1 - \frac{r}{c}) & \text{if } c \leq r
  \end{cases}
$

The researchers used a baseline of $N = 4, w_n = \frac{1}{N} = \frac{1}{4}$

#Implementation

## $\text{Count}$ and $\text{Count}_{clip}$

First, we'll define some functions to calculate the $\text{Count}$ and $\text{Count}_{clip}$:

In [12]:
import re
import numpy as np

### n_gram: string representing an n-gram
### translation: list strings of all n-grams in the machine translation
def count(n_gram, translation):
  return np.sum(np.array(translation) == n_gram)

### n_gram: string representing an n-gram
### refs: list of lists containing strings, the n-grams in each reference
def count_ref(n_gram, refs):
  max_count = 0
  for ref in refs:
    rcount = np.sum(np.array(ref) == n_gram)
    if rcount > max_count:
      max_count = rcount
  return max_count

### n_gram: string representing an n-gram 
### translation: list of all n-grams in the machine translation
### refs: list of lists containing strings, the n-grams in each reference
def count_clip(n_gram, translation, refs):
  return min(count(n_gram, translation), count_ref(n_gram, refs))

## $p_n$ for single sentences

Now, let's code a function to calculate $p_n$ for single sentence translations and then we'll generalize entire translated documents.

In [317]:
# calculates pn for single sentences

### translation: a string machine translation
### refs: a list of strings of reference translations
### n: int, n-gram length 
def pn_single(translation, refs, n):

  translation = translation.lower() # don't care about capitalization 
  translation = re.sub(r'[^\w\s]', '',translation) # remove punctuation
  translation = translation.split(' ') # split at spaces
  
  translation_n_grams = []

  #creating list of n-grams in translation
  for i in range(len(translation) - n + 1):
    translation_n_grams.append(' '.join(translation[i : i + n]))

  #processing reference sentences
  for idx, ref in enumerate(refs):
    ref = ref.lower() 
    ref = re.sub(r'[^\w\s]', '',ref) 
    refs[idx] = ref.split(' ')
  
  ref_n_grams_full = []
  #creating list of lists of n-grams in the references
  for ref in refs:
    ref_n_grams = []
    for i in range(len(ref) - n + 1):
      ref_n_grams.append(' '.join(ref[i : i + n]))
    ref_n_grams_full.append(ref_n_grams)
  
  numerator = 0
  denominator = 0

  evaluated_n_grams = []

  for ng in translation_n_grams:
    if ng not in evaluated_n_grams: # we don't want to evaluate the same n-gram twice, but we want to count all occurrences
      numerator += count_clip(ng, translation_n_grams, ref_n_grams_full)
      denominator += count(ng, translation_n_grams)
      evaluated_n_grams.append(ng)

  return numerator/denominator

In [318]:
machine_translation = 'The cat the cat the cat.'
references = ['The cat is on the mat.', 'There is a cat on the mat']

pn_single(machine_translation, references, 1)

0.5

That returns the correct value we calculated earlier!

## $p_n$ for multi-sentence strings

Now I'll implement the generalized version for multi-sentence documents. First I'm going to create an n-gram helper. This just returns the lists containing the n-grams for the translation and references just like for `pn_single` but it is slightly more nested and ugly. 

I didn't have to make a helper function and could have just left it inside the `pn` function, but I thought I would need this for the brevity penalty, although it turned out I did not!

In [265]:
# Returns lists of n-grams for the translations and references

def n_gram_helper(translations, refs_full, n):
  t = translations.lower() # don't care about capitalization 
  t = t.split('.')
  t = t[:-1] # to remove a trailing '' in list

  for idx, translation in enumerate(t): 
    translation = translation.strip()
    translation = re.sub(r'[^\w\s]', '',translation) # remove punctuation
    translation = translation.split(' ') # split at spaces
    t[idx] = translation
  
  translation_n_grams_full = []
  for translation in t:
    translation_n_grams = []
    for i in range(len(translation) - n + 1):
      translation_n_grams.append(' '.join(translation[i : i + n]))
    translation_n_grams_full.append(translation_n_grams)

  #processing reference sentences
  r_full = []
  for idx1, refs in enumerate(refs_full): 
    rs = []
    for idx2, ref in enumerate(refs):
      r = ref.lower() 
      r = re.sub(r'[^\w\s]', '',r) 
      rs.append(r.split(' '))
    r_full.append(rs)
  
  ref_n_grams_full = []
  for refs in r_full:
    ref_n_grams_mid = []
    for ref in refs:
      ref_n_grams = []
      for i in range(len(ref) - n + 1):
        ref_n_grams.append(' '.join(ref[i : i + n]))
      ref_n_grams_mid.append(ref_n_grams)
    ref_n_grams_full.append(ref_n_grams_mid)

  return translation_n_grams_full, ref_n_grams_full

In [241]:
#calculates pn for entire documents, returns both pn and brevity penalty for fewer computations.

### translation: string containing all translated sentences
### refs: list of lists containing the possible reference translations for each sentence in order. 
### n : int, n-gram length. 
def pn(translations, refs_full, n):
  translation_n_grams_full, ref_n_grams_full = n_gram_helper(translations, refs_full, n)
  # print(translation_n_grams_full)
  # print(ref_n_grams_full)

  numerator = 0
  denominator = 0

  for idx, translation in enumerate(translation_n_grams_full):
    evaluated_n_grams = []
    for ng in translation:
      if ng not in evaluated_n_grams: # we don't want to evaluate the same n-gram twice, but we want to count all occurrences
        numerator += count_clip(ng, translation, ref_n_grams_full[idx])
        denominator += count(ng, translation)
        evaluated_n_grams.append(ng)

  return numerator/denominator

If this works then according to the formula, the correct pn for two sentences that are identical to our earlier example should be 
$(3 + 3)/(6 + 6) = .5$ still. 

In [224]:
test = 'The cat the cat the cat. The cat the cat the cat.'
refs = [['The cat is on the mat.', 'There is a cat on the mat'], ['The cat is on the mat.', 'There is a cat on the mat']]
pn(test, refs, 1)

0.5

And the following example should yield (2 + 3) / (7 + 6)



In [82]:
test = 'The the the the the the the. The cat the cat the cat.'
refs = [['The cat is on the mat.', 'There is a cat on the mat'], ['The cat is on the mat.', 'There is a cat on the mat']]
pn(test, refs, 1)[0]

0.38461538461538464

In [83]:
5/13

0.38461538461538464

Smashing, now we can implement the brevity penalty. 

## Brevity Penalty

We just need to calculate the length of the translation and for each sentence find the reference that's closest in length. 

In [319]:
# returns brevity penalty

### translation: list of all the words in a translation
### refs: list of lists containing all words in a reference translation
def brevity_penalty(translation, refs):
  c = 0 
  r = 0
  sentences = translation.split('.')
  sentences.remove('')
  for idx, s in enumerate(sentences):
    s_split = s.strip().split(' ')
    t_sentence_length = len(s_split)
    c += t_sentence_length # sum translation lengths 
    
    # finding the best reference length
    best_ref_length = float('inf') # start at infinity so first reference satisfies inequality below
    for ref in refs[idx]:
      if abs(len(ref.strip().split(' ')) - t_sentence_length) < abs(t_sentence_length - best_ref_length): # Check distance between new and old ref length - translation 
        best_ref_length = len(ref.strip().split(' '))

    r += best_ref_length # summing best ref lengths to get effective ref length, r

  if c > r:
    return 1
  else:
    return np.exp(1 - r/c)

To check that it works I'm going to do an example where the translation length exactly matches the reference lengths -> bp = 1, and an example that's shorter -> bp < 1

In [320]:
# Should yield BP = 1
test = 'The cat the cat the cat. There is not a dog door.'
refs = [['The cat is on the mat', 'There is a cat on the mat.'], ['The dog is at the door', 'There is a dog at the door']]

brevity_penalty(test, refs)

1.0

In [323]:
# Should yield BP < 1
test = 'cat. dog.'
refs = [['The cat is on the mat', 'There is a cat on the mat.'], ['The dog is at the door', 'There is a dog at the door']]

brevity_penalty(test, refs)

0.006737946999085467

Very good, the second example is *much* shorter than the references so thankfully the bp is quite small. 


## BLEU Score
Finally, let's put it all together and get a working BLEU score:

In [315]:
#Calculates BLEU score with uniform weights

def BLEU_score(translation, refs, N, wn):   
  sum = 0
  bp = brevity_penalty(translation, refs)

  for n in range(1, N + 1):
    p = pn(translation, refs, n)
    sum += wn * np.log(p)
    
  return bp * np.exp(sum)


In [316]:
test = 'The cat the cat the cat. There is a dog door.'
refs = [['The cat is on the mat', 'There is a cat on the mat.'], ['The dog is at the door', 'There is a dog at the door']]

BLEU_score(test, refs, 4, .25)

0.3366184122712009

There we go, we have a score and since the first sentence is almost entirely wrong - it makes sense the score is pretty low. 

#Conclusions

I'm actually a bit iffy on the current state of my implementation. Going through the paper for the BLEU scoring system was a bit of a rollercoaster - the notation was hard to keep clear for me and I was originally uncomfortable with it, then after reading it again and starting implementation thought I had figured it out, but now I'm uncomfortable again, here's why: 

## Uh oh...

What if we set N = 5? Well, there's no 5-gram in either sentence that matches any reference, so $p_5 = 0$ must be true. But if $p_5 = 0$, then we end up adding $\log(0)$ to the sum, well... I think we all know that's not going to work out. 

On its own, if we got $e^{\log(0)}$ that would be fine, because it would make sense, if $p_n = 0$  then of course our idea of accuracy should be 0. But using the example above that's not quite the case because we have some level of accuracy for 1,2,3, and 4-grams. 

## Why might this be happening?

At the moment I am unsure if my interpretation of their notation / formulas is incorrect, or if this is an intended behavior of the scoring system, perhaps this is why they only did up to 4-grams as their baseline, because if there's any sentence with fewer than $N$ n-grams then the BLEU score will always be 0.

I thought it may have been my $p_n$ code since it's the most involved, but after running this on some more complex examples directly from the paper I'm actually getting the same values as the researchers! So that may not be the case.

I'm still leaning towards my understanding being wrong, which is unfortunate, but less catastrophic than the thought of BLEU having such a major flaw. If you spot what I did wrong, please let me know! This will keep me up for nights to come.... 

## Final Remarks

I was hoping to load a translation model and mess around to see how well my scores lined up with scores in literature, but its quite late so that will not be happening this week!

Also, as a final note, don't use my implementation if you want to do BLEU scoring on a translation model - there are more efficient libraries online and I was just implementing as a way to become more familiar with the scoring system. 
